In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import glob, tqdm

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import PIL.Image as Image

In [5]:
model_name = 'cycle_gan'
model_sig  = 'bayer'
base_dir  = '/home/team01/datasets/mit'

mytype = 'train'
# mytype = 'val'
mytype = 'test'


## find trining dataset
src_dir = os.path.join(base_dir, 'images', mytype)
folders = glob.glob(os.path.join(src_dir, '**', '**'))
files = []
folders.sort()
print(src_dir)
print(folders)
for temp in folders:
    folder_num = temp.split('/')[-1]
    if mytype == 'train':
        print(temp, folder_num)
        if int(folder_num) % 10 == 1:
            print(temp)
            files += glob.glob(os.path.join(temp, '**/**.png'), recursive=True)
    else:
        print(temp)
        files += glob.glob(os.path.join(temp, '**/**.png'), recursive=True)
        

print(files[-1])

/home/team01/datasets/mit/images/test
['/home/team01/datasets/mit/images/test/hdrvdp/000', '/home/team01/datasets/mit/images/test/moire/000']
/home/team01/datasets/mit/images/test/hdrvdp/000
/home/team01/datasets/mit/images/test/moire/000
/home/team01/datasets/mit/images/test/moire/000/000694.png


In [6]:
dirs = os.listdir(src_dir)
for idx, d in enumerate(dirs):
    print(idx, d)

0 filelist.txt
1 hdrvdp
2 moire


In [7]:
# target folders
new_name = 'images' + '_'  +  model_name + '_' + model_sig
target_dir = os.path.join(base_dir,new_name, mytype)
os.makedirs(target_dir, mode=777, exist_ok=True)

In [8]:
target_dir

'/home/team01/datasets/mit/images_cycle_gan_bayer/test'

In [9]:
## model
print(src_dir)
print(target_dir)

name_structure = os.path.join('model_dir', 'checkpoint', model_name + '_G_model_structure.h5')
ckpt_path = os.path.join('model_dir', 'checkpoint',  model_name+'_'+model_sig )
print(name_structure)
print(ckpt_path)
checkpoints = glob.glob(os.path.join(ckpt_path, '*.h5'))
checkpoints.sort()

/home/team01/datasets/mit/images/test
/home/team01/datasets/mit/images_cycle_gan_bayer/test
model_dir/checkpoint/cycle_gan_G_model_structure.h5
model_dir/checkpoint/cycle_gan_bayer


In [10]:
model = tf.keras.models.load_model(name_structure, custom_objects={'tf':tf, "InstanceNormalization":tfa.layers.InstanceNormalization})
model.load_weights(checkpoints[-1])
model.summary()
print(checkpoints[-1])

Model: "generator_G"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
generator_G_img_input (InputLay [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 9408        generator_G_img_input[0][0]      
__________________________________________________________________________________________________
instance_normalization (Instanc (None, 128, 128, 64) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           instance_normalization[0][0]     
________________________________________________________________________________________

In [11]:
print(checkpoints[-1])

model_dir/checkpoint/cycle_gan_bayer/G_00523_cycle_gan_bayer_7.82073e-01.h5


In [12]:
# inference & save
ntot = len(files)
for idx, f in tqdm.tqdm(enumerate(files)):
    name = f.split(mytype)[-1]
    fsave = os.path.join(target_dir, name[0:])
    fsave = target_dir + name[:-4]
#     print('---->', name)
#     print('====>', fsave)
#     print('=-=->', target_dir)

#     break

    image = Image.open(f)
    arr = np.array(image)
#     print(arr.shape, np.amin(arr), np.amax(arr))

    # normalize
    arr = arr.astype(np.float32) / 255.
    arr = arr*2 -1

    # inference
    pred = model.predict(arr[np.newaxis,...]) # (-1, 1) --> (-1, 1)

    # expand (-1, 1) -> (0, 65535)
    pred = pred[0]
    pred = (pred +1) / 2 # (-1, 1) -> (0, 1)
    pred = pred * 65535
    pred = pred.astype(np.uint16)
#     print('>>>>> ',fsave[:-7])
#     print(pred.shape, np.amin(pred), np.amax(pred), pred.dtype)

    os.makedirs(fsave[:-7], mode=777, exist_ok=True)
    np.save(fsave, pred)


#     break

2000it [02:13, 15.00it/s]


In [28]:
!pip install tqdm